In [4]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [5]:
! pip install mamba-ssm

In [2]:
import torch
print(torch.__version__)

2.4.1+cu121


In [3]:
import argparse
import os.path as osp
from typing import Any, Dict, Optional

import torch
from torch.nn import (
    BatchNorm1d,
    Embedding,
    Linear,
    ModuleList,
    ReLU,
    Sequential,
)
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch_geometric.transforms as T
from torch_geometric.datasets import ZINC
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINEConv, global_add_pool
import inspect
from typing import Any, Dict, Optional

import torch.nn.functional as F
from torch import Tensor
from torch.nn import Dropout, Linear, Sequential

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.inits import reset
from torch_geometric.nn.resolver import (
    activation_resolver,
    normalization_resolver,
)
from torch_geometric.typing import Adj
from torch_geometric.utils import to_dense_batch

from mamba_ssm import Mamba
from torch_geometric.utils import degree, sort_edge_index
import torch_sparse
#import torch_geometric

/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:1045: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type=

In [12]:
!pip uninstall torch-sparse torch-geometric torch-scatter torch-cluster

Found existing installation: torch_sparse 0.6.18
Uninstalling torch_sparse-0.6.18:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torch_sparse-0.6.18.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch_sparse/*
Proceed (Y/n)? Y
  Successfully uninstalled torch_sparse-0.6.18
Found existing installation: torch_geometric 2.3.1
Uninstalling torch_geometric-2.3.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torch_geometric-2.3.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch_geometric/*
Proceed (Y/n)? Y
  Successfully uninstalled torch_geometric-2.3.1
Found existing installation: torch_scatter 2.1.2
Uninstalling torch_scatter-2.1.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torch_scatter-2.1.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch_scatter/*
Proceed (Y/n)? Y
  Successfully uninstalled torch_scatter-2.1.2


In [13]:
import torch
print(torch.__version__)

2.4.1+cu121


In [14]:
!pip install torch-sparse torch-scatter torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.4.1+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.4.1+cu121.html
  Using cached torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl
  Using cached torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl
  Using cached torch_cluster-1.6.3.tar.gz (54 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_spline_conv-1.2.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.2 MB/s eta 0:00:00
  Created wheel for torch-cluster: filename=torch_cluster-1.6.3-cp310-cp310-linux_x86_64.whl size=731660 sha256=9428de47be15966f2647f55cf254fd881335f7544aa403da229cb5294f525af8
  Stored in directory: /root/.cache/pip/wheels/51/78/c3/536637b3cdcc3313aa5e8851a6c72b97f6a01877e68c7595e3
  Created wheel for torch-spline-conv: filename=torch_spline_conv-1.2.2-cp310-cp310-linux_x86_64.whl size=225391 sha256=83b09eb4c7ec360c35e4c60d1e82b8705428499ae2115f911dfbe1c086a

In [6]:
path, subset = '/scratch/ssd004/scratch/tsepaole/ZINC_full/', False
path, subset = '', True

transform = T.AddRandomWalkPE(walk_length=20, attr_name='pe')
train_dataset = ZINC(path, subset=subset, split='train', pre_transform=transform)
val_dataset = ZINC(path, subset=subset, split='val', pre_transform=transform)
test_dataset = ZINC(path, subset=subset, split='test', pre_transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

Processing...
Processing test dataset: 100%|██████████| 1000/1000 [00:00<00:00, 1107.82it/s]
Done!


In [7]:
class GPSConv(torch.nn.Module):

    def __init__(
        self,
        channels: int,
        conv: Optional[MessagePassing],
        heads: int = 1,
        dropout: float = 0.0,
        attn_dropout: float = 0.0,
        act: str = 'relu',
        att_type: str = 'transformer',
        order_by_degree: bool = False,
        shuffle_ind: int = 0,
        d_state: int = 16,
        d_conv: int = 4,
        act_kwargs: Optional[Dict[str, Any]] = None,
        norm: Optional[str] = 'batch_norm',
        norm_kwargs: Optional[Dict[str, Any]] = None,
    ):
        super().__init__()

        self.channels = channels
        self.conv = conv
        self.heads = heads
        self.dropout = dropout
        self.att_type = att_type
        self.shuffle_ind = shuffle_ind
        self.order_by_degree = order_by_degree

        assert (self.order_by_degree==True and self.shuffle_ind==0) or (self.order_by_degree==False), f'order_by_degree={self.order_by_degree} and shuffle_ind={self.shuffle_ind}'
        # order_by_degree flag is set to True, this section reorders the nodes based on their degrees
        if self.att_type == 'transformer':
            self.attn = torch.nn.MultiheadAttention(
                channels,
                heads,
                dropout=attn_dropout,
                batch_first=True,
            )
        if self.att_type == 'mamba':
            self.self_attn = Mamba(
                d_model=channels,
                d_state=d_state,
                d_conv=d_conv,
                expand=1
            )

        self.mlp = Sequential(
            Linear(channels, channels * 2),
            activation_resolver(act, **(act_kwargs or {})),
            Dropout(dropout),
            Linear(channels * 2, channels),
            Dropout(dropout),
        )

        norm_kwargs = norm_kwargs or {}
        self.norm1 = normalization_resolver(norm, channels, **norm_kwargs)
        self.norm2 = normalization_resolver(norm, channels, **norm_kwargs)
        self.norm3 = normalization_resolver(norm, channels, **norm_kwargs)

        self.norm_with_batch = False
        if self.norm1 is not None:
            signature = inspect.signature(self.norm1.forward)
            self.norm_with_batch = 'batch' in signature.parameters

    def reset_parameters(self):
        r"""Resets all learnable parameters of the module."""
        if self.conv is not None:
            self.conv.reset_parameters()
        self.attn._reset_parameters()
        reset(self.mlp)
        if self.norm1 is not None:
            self.norm1.reset_parameters()
        if self.norm2 is not None:
            self.norm2.reset_parameters()
        if self.norm3 is not None:
            self.norm3.reset_parameters()

    def forward(
        self,
        x: Tensor,
        edge_index: Adj,
        batch: Optional[torch.Tensor] = None,
        **kwargs,
    ) -> Tensor:
        r"""Runs the forward pass of the module."""
        hs = []
        if self.conv is not None:  # Local MPNN.
            h = self.conv(x, edge_index, **kwargs)
            h = F.dropout(h, p=self.dropout, training=self.training)
            h = h + x
            if self.norm1 is not None:
                if self.norm_with_batch:
                    h = self.norm1(h, batch=batch)
                else:
                    h = self.norm1(h)
            hs.append(h)

        ### Global attention transformer-style model.
        if self.att_type == 'transformer':
            h, mask = to_dense_batch(x, batch)
            h, _ = self.attn(h, h, h, key_padding_mask=~mask, need_weights=False)
            h = h[mask]

        if self.att_type == 'mamba':

            if self.order_by_degree:
              # degree() function: edge_index[0]= source nodes of the edges, x.shape[0]= number of nodes)
                deg = degree(edge_index[0], x.shape[0]).to(torch.long)
                order_tensor = torch.stack([batch, deg], 1).T
                # creates a tensor by stacking the 'batch' tensor(batch_index of each node) and the degree of each node. making it easier to reorder nodes
                _, x = sort_edge_index(order_tensor, edge_attr=x)
                # reorders the feature tensor x based on the node degree and batch information, reordering x as well.

            if self.shuffle_ind == 0:
              # no shuffling occurs, and dense batch repr. is created from x
                h, mask = to_dense_batch(x, batch)
                h = self.self_attn(h)[mask]
                # the result is masked to remove invalid entries (padded entries)
                #
            else:
                mamba_arr = []
                # list to store the results of multiple attention passes over shuffled node features
                for _ in range(self.shuffle_ind):
                  # loops over the number of shuffles(shuffle_ind)
                    h_ind_perm = permute_within_batch(x, batch)
                    # Similar to before, converts the shuffled x into a dense representation for each bat
                    h_i, mask = to_dense_batch(x[h_ind_perm], batch)
                    # Similar to before, converts the shuffled x into a dense representation for each batch.
                    h_i = self.self_attn(h_i)[mask][h_ind_perm]
                    # The shuffled dense batch is passed through the self-attention mechanism, and the result is reordered back to the original node order (h_ind_perm).
                    mamba_arr.append(h_i)
                h = sum(mamba_arr) / self.shuffle_ind

                # Averages the results from all shuffled versions to produce the final node representations.
        ###

        h = F.dropout(h, p=self.dropout, training=self.training)
        h = h + x  # Residual connection.
        if self.norm2 is not None:
            if self.norm_with_batch:
                h = self.norm2(h, batch=batch)
            else:
                h = self.norm2(h)
        hs.append(h)

        out = sum(hs)  # Combine local and global outputs.

        out = out + self.mlp(out)
        if self.norm3 is not None:
            if self.norm_with_batch:
                out = self.norm3(out, batch=batch)
            else:
                out = self.norm3(out)

        return out

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.channels}, '
                f'conv={self.conv}, heads={self.heads})')

In [9]:
class GraphModel(torch.nn.Module):
    def __init__(self, channels: int, pe_dim: int, num_layers: int, model_type: str, shuffle_ind: int, d_state: int, d_conv: int, order_by_degree: False):
        super().__init__()

        # pe_dim: Dimensionality of the positional encoding
        # num_layers = ?
        # shuffle_ind: number of shuffle indices for the Mamba attention mechanism
        # d_state, d_conv: parameters of Mamba attention
        # order_by_degree: If nodes should be ordered by degree

        self.node_emb = Embedding(28, channels - pe_dim)
        # An embedding layer that maps node input features into the space of channels - pe_dim dimensions.
        self.pe_lin = Linear(20, pe_dim)
        # A linear layer that transforms the positional encoding (pe) from 20 dimensions to pe_dim.
        self.pe_norm = BatchNorm1d(20)
        # Batch normalization applied to the positional encoding to normalize it.
        self.edge_emb = Embedding(4, channels)
        # Embedding for edge attributes, mapping them from 4 features into the same space as the nodes (channels).
        self.model_type = model_type
        self.shuffle_ind = shuffle_ind
        self.order_by_degree = order_by_degree

        # Graph Convolution layers
        self.convs = ModuleList()
        # A list of graph convolutional layers

        # Creates a two-layer neural network (nn), used within each GINEConv layer
        for _ in range(num_layers):
            nn = Sequential(
                Linear(channels, channels),
                ReLU(),
                Linear(channels, channels),
            )
            if self.model_type == 'gine':
                conv = GINEConv(nn)

            if self.model_type == 'mamba':
                conv = GPSConv(channels, GINEConv(nn), heads=4, attn_dropout=0.5,
                               att_type='mamba',
                               shuffle_ind=self.shuffle_ind,
                               order_by_degree=self.order_by_degree,
                               d_state=d_state, d_conv=d_conv)

            if self.model_type == 'transformer':
                conv = GPSConv(channels, GINEConv(nn), heads=4, attn_dropout=0.5, att_type='transformer')

            # conv = GINEConv(nn)
            self.convs.append(conv)

        self.mlp = Sequential(
            Linear(channels, channels // 2),
            ReLU(),
            Linear(channels // 2, channels // 4),
            ReLU(),
            Linear(channels // 4, 1),
        )

    def forward(self, x, pe, edge_index, edge_attr, batch):
      # x: Node attributes
      # pe: Positional encoding for nodes
      # edge_index: edge indices of the graph
      # edge_attr: edge attributes
      # batch
        x_pe = self.pe_norm(pe)
        # Batch normalization to positional encodings

        x = torch.cat((self.node_emb(x.squeeze(-1)), self.pe_lin(x_pe)), 1)
        # embedding applied to the node features
        # linear transformation applied to x_pe
        # concatenate along feature dimension

        edge_attr = self.edge_emb(edge_attr)
        # edge embedding applied to edge attributes, transforms into the same dim. as nodes

        for conv in self.convs:
            if self.model_type == 'gine':
                x = conv(x, edge_index, edge_attr=edge_attr)
            else:
                x = conv(x, edge_index, batch, edge_attr=edge_attr)

        x = global_add_pool(x, batch)
        # node features are pooled at the graph level using global pooling
        return self.mlp(x)

In [ ]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.pe, data.edge_index, data.edge_attr,
                    data.batch)
        loss = (out.squeeze() - data.y).abs().mean()
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
    return total_loss / len(train_loader.dataset)

In [ ]:
@torch.no_grad()
def test(loader):
    model.eval()

    total_error = 0
    for data in loader:
        data = data.to(device)
        # print(data.x.shape)
        out = model(data.x, data.pe, data.edge_index, data.edge_attr,
                    data.batch)
        total_error += (out.squeeze() - data.y).abs().sum().item()
    return total_error / len(loader.dataset)

In [ ]:
# it.to(device)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GraphModel(channels=64, pe_dim=8, num_layers=10,
                   model_type='mamba',
                   shuffle_ind=0, order_by_degree=True,
                   d_conv=4, d_state=16,
                  ).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20,
                              min_lr=0.00001)
arr = []
for epoch in range(1, 30):
    loss = train()
    val_mae = test(val_loader)
    test_mae = test(test_loader)
    scheduler.step(val_mae)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_mae:.4f}, '
          f'Test: {test_mae:.4f}')
    arr.append(test_mae)
ordering = arr
print(ordering)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GraphModel(channels=64, pe_dim=8, num_layers=10,
                   model_type='mamba',
                   shuffle_ind=1, order_by_degree=False,
                   d_conv=4, d_state=16,
                  ).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20,
                              min_lr=0.00001)
arr = []
for epoch in range(1, 30):
    loss = train()
    val_mae = test(val_loader)
    test_mae = test(test_loader)
    scheduler.step(val_mae)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_mae:.4f}, '
          f'Test: {test_mae:.4f}')
    arr.append(test_mae)
permute = arr
print(permute)

In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd

# import numpy as np
# res_df = pd.read_csv('30_ep_res.csv')

# WINDOW = 1
# fig, ax = plt.subplots(1, figsize=(15,5))

# for col in res_df.columns:
#     plt.plot(res_df[col].clip(0,0.7).rolling(WINDOW, min_periods=1).mean(), label=col)

# plt.legend()
# plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, figsize=(15,5))

plt.plot(permute[:20], label='permute')
plt.plot(ordering[:20], label='order')

plt.legend()
plt.show()

In [ ]:
fffffffffff

# Plotting

# Tests

In [ ]:
ggggggggggg

In [ ]:
it = next(iter(train_loader))
# h, mask = to_dense_batch(it.x, it.batch)
# it.x.shape, h.shape, mask.shape

In [ ]:
deg.dtype

In [ ]:
it

In [ ]:
it.edge_index[0]

In [ ]:
it.to(device)
out = model(it.x, it.pe, it.edge_index, it.edge_attr,
                    it.batch)

In [ ]:
batch = torch.tensor([0,0,0,1,1,1,1])
x = torch.tensor([0,1,2,3,4,5,6])
batch.shape, x.shape

In [ ]:
import torch

def permute_within_batch(x, batch):
    # Enumerate over unique batch indices
    unique_batches = torch.unique(batch)

    # Initialize list to store permuted indices
    permuted_indices = []

    for batch_index in unique_batches:
        # Extract indices for the current batch
        indices_in_batch = (batch == batch_index).nonzero().squeeze()

        # Permute indices within the current batch
        permuted_indices_in_batch = indices_in_batch[torch.randperm(len(indices_in_batch))]

        # Append permuted indices to the list
        permuted_indices.append(permuted_indices_in_batch)

    # Concatenate permuted indices into a single tensor
    permuted_indices = torch.cat(permuted_indices)

    return permuted_indices

# Example usage
batch = torch.tensor([0, 0, 0, 1, 1, 1, 1])
x = torch.tensor([0, 10, 20, 30, 40, 50, 60])

# Get permuted indices
permuted_indices = permute_within_batch(x, batch)

# Use permuted indices to get the permuted tensor
permuted_x = x[permuted_indices]

print("Original x:", x)
print("Permuted x:", permuted_x)
print("Permuted indices:", permuted_indices)

In [ ]:
mask[0].sum(), (it.batch==0).sum()

In [ ]:
self_attn = Mamba(d_model=64, # Model dimension d_model
                                d_state=16,  # SSM state expansion factor
                                d_conv=4,    # Local convolution width
                                expand=1,    # Block expansion factor
                            )
print(sum(p.numel() for p in self_attn.parameters() if p.requires_grad), sum(p.numel() for p in self_attn.parameters()))

In [ ]:
self_attn = Mamba(d_model=64, # Model dimension d_model
                                d_state=8,  # SSM state expansion factor
                                d_conv=2,    # Local convolution width
                                expand=1,    # Block expansion factor
                            )
print(sum(p.numel() for p in self_attn.parameters() if p.requires_grad), sum(p.numel() for p in self_attn.parameters()))

In [ ]:
self_attn = Mamba(d_model=64, # Model dimension d_model
                                d_state=16,  # SSM state expansion factor
                                d_conv=8,    # Local convolution width
                                expand=1,    # Block expansion factor
                            )
print(sum(p.numel() for p in self_attn.parameters() if p.requires_grad), sum(p.numel() for p in self_attn.parameters()))

In [ ]:
self_attn = torch.nn.MultiheadAttention(
                64,
                4,
                dropout=0.5,
                batch_first=True,
            )
print(sum(p.numel() for p in self_attn.parameters() if p.requires_grad), sum(p.numel() for p in self_attn.parameters()))